# MasterChessMind - Project Intro
What are some of the most significates dates in the history of computer science?

This notebook was created by Yuval Krinsky as part of the Data Science workshop at the Open University of Israel, Spring 

## Chess in the advancements of Artificial Intelligence
Throughout history, chess was considered an indicator of wisdom and one who was good at it, was regarded as a smart and wise man. 
These days, chess is one of the world's most popular board games, played and admired for its strategic depth and intellectual challenge.

Chess players have dedicated years and decades in improving their strategies and tactics in the game. 
And as chess was considered a test of intellectual superiority, it's no suprise computer scientists took on the challenge of developing a computer program which will be able to play and defeat the best of the best human players.

Therefore, when in 1997, the **Deep Blue super computer defeated the current world champion and the chess legend Garry Kasparov**, it was and still considered a major milestone in the development of the field of AI in computer science.
The news of the defeat spread like fire in the whole world, and Deep Blue is even the subject of a few movies and books.
It important to mention that the task of building Deep Blue was sure not a piece of cake, and the computer giant IBM has invested more than 10 years in it's development!

![AI Timeline](./res/AI_timeline.png)

## Personal Connection
I myself is a bit of chess fan, and doesn't miss the chance to play against a friend or a family member.
I also used my love for the game to teach volenturylly children the game, to empower them through the game, with a non-profit organization called cpchess.

And from a personal perspective, I withstand the complexity of the game and the high skills needed to be a good player.
Therefore I thought of this as a great example to show the power of AI and specifically ML.

## Project Goal
The project goal is to provide a smart AI assistant for chess games. Currently, I don't have a real use of it in the world, but it's a very good demonstration of the advancements we can use AI and ML for.

The project is divided into 2 big parts:
### Chess Computer Vision
The first part of the program will be responsible for analyzing an image of a chess board and giving as an output a formal representation which can be understood easily by the computer and will be used in the second part of the project.
### Position Evaluation
The second part of the project will be to evaluate all the different moves the player can do from this position, and give the user a very good move he can do.

Both parts are very hard and complex to develop without using ML, and I hope that by using ML tools, we could develop a much simpler solution which will still give good results.

***

# Developing Position Evaluation
As a reminder, this part of the project goal is - 

**Given a board position, evaluate which player is leading and by how much!**

This is probably the most complex and difficult problem in chess, as there are many complex factors which should be taken in to consideration in such evaluation, ranging from remaining pieces strength and positions to king safety and bishop pair advantages.

And the **proposed solution for this problem is building a deep neural network, which will take as an input the board position, and will return the estimated evaluation**.

## Evaluation Intro
There is a standard representation for chess board evaluations. Given a chess position, there is a standard to give a number which indicates who leads and by how much.

If the white player is leading, this number will be positive and sometime explicitly marked with a '+' as a prefix. If the black player is leading, the evaluation will be negative and will start with '-'.

Generally, different pieces are worth different points for evaluating. For example, a pawn (the most basic piece) is generally worth 1 point, while the rook a much stronger piece worth 5 points, and the queen, which is the most important piece except the king, is worth 9 points.
This is just the most basic idea for evaluating. In practice, many more factors should be factored in, such as the pieces positions, the positions related to other pieces, relation to other pieces remaining on the board and more.

But to summarize, an evaluation of '+1.42' means the white player is in the lead but not in such a big gap. And an evaluation of '-9.12' means the black player lead by a lot, and the game is pretty much over from here.

![Pieces Evaluations](./res/pieces_evaluations.png)

## Exploratory Data Analysis
I've used Kaggle and found a very big [dataset](https://www.kaggle.com/datasets/ronakbadhe/chess-evaluations), which maps board positions to their corresponding evaluation as calculated by Stockfish (a famous and very highly complex open source chess engine).

From an inspection of the dataset 3 problems arose:
* Board representation
* Checkmate positions
* Overweighed data

### Board Representation
Our dataset looks like this:
```Csv
N1bk2nr/pp1p1ppp/8/4P3/8/8/P1qNPPPP/R3KB1R w KQ - 0 12,-409
r2k1bnr/2Np1ppp/b1n1p3/8/4P3/P4N2/1PP2PPP/R1B1K2R w KQ - 3 12,+382
8/6k1/pp1p3p/3P2p1/2PN1rP1/1P6/P6P/6K1 w - - 1 39,+431
```
The first field is the board representation, and the second value is the board evaluation (in centipawns units, which means every pawn is equal to 100 and not 1).

As you can clearly see, the board representation isn't something we can directly use in building a neural network.
This representation is actually a standard representation called FEN. FEN (Forsyth-Edwards Notation) is a concise text-based format used to describe the current state of a chessboard position, it includes the pieces positions and more advanced parts of the game which are needed to exactly describe the current state of the game.

We need to convert this representation to something we could use as the input layer for our neural network.
After learning and understanding this notation, I chose the important features for evaluating the board position (pieces placement and castling rights), and wrote a function that given a FEN string, builds a PyTorch tensor representing the board.

A chess board consists of 64 (8x8) squares, and we have 12 different pieces types (6 white and 6 black). Therefore I chose to represent the pieces placement with a long array of boolean bits. For each piece type, we have 64 bits, indicating if the piece is currently on the corresponding square.
For castling rights, I used 4 bits. And for which player turn it right now, one more bit.
In total, we have 773 (64*12+4+1) bits for representing the board.

In [1]:
import torch

def fen_to_bitboard_tensor(fen: str):
    # Initialize bitboards for different piece types
    empty = 0
    white_pawns = 0
    white_knights = 0
    white_bishops = 0
    white_rooks = 0
    white_queens = 0
    white_kings = 0
    black_pawns = 0
    black_knights = 0
    black_bishops = 0
    black_rooks = 0
    black_queens = 0
    black_kings = 0

    # Split FEN into parts
    fen_board_part, fen_turn_part, fen_castling_part, *_ = fen.split()

    # Translate FEN board representation to bitboards

    # Start from the 8th rank and a-file (0-based index)
    row = 7  
    col = 0 

    for char in fen_board_part:
        if char == '/':
			# Continue to next line
            row -= 1
            col = 0
        elif char.isnumeric():
        	# Numeric means empty spaces, so we skip accordingly to the next piece in row.
            col += int(char)
        else:
        	# Convert rank and file to a square index
            square = row * 8 + col 

            if char == 'P':
                white_pawns |= 1 << square
            elif char == 'N':
                white_knights |= 1 << square
            elif char == 'B':
                white_bishops |= 1 << square
            elif char == 'R':
                white_rooks |= 1 << square
            elif char == 'Q':
                white_queens |= 1 << square
            elif char == 'K':
                white_kings |= 1 << square
            elif char == 'p':
                black_pawns |= 1 << square
            elif char == 'n':
                black_knights |= 1 << square
            elif char == 'b':
                black_bishops |= 1 << square
            elif char == 'r':
                black_rooks |= 1 << square
            elif char == 'q':
                black_queens |= 1 << square
            elif char == 'k':
                black_kings |= 1 << square

            col += 1

    pieces = [
    	white_pawns, white_knights, white_bishops, white_rooks, white_queens, white_kings, 
    	black_pawns, black_knights, black_bishops, black_rooks, black_queens, black_kings
    ]
    pieces_board_bits = []
    for piece in pieces:
    	# Pad to 64 bits for each piece and skip the '0b' prefix
    	piece_bits_str = bin(piece)[2:].zfill(64)
    	piece_bits = [int(bit) for bit in piece_bits_str]

    	pieces_board_bits.extend(piece_bits)

    # Determine player turn
    turn_bits = [1 if fen_turn_part == 'w' else 0]

    # Determine castling rights
    in_castling = lambda x: 1 if x in fen_castling_part else 0
    white_kingside_castle = in_castling('K')
    white_queenside_castle = in_castling('Q')
    black_kingside_castle = in_castling('k')
    black_queenside_castle = in_castling('q')

    castling_bits = [white_kingside_castle, white_queenside_castle, black_kingside_castle, black_queenside_castle]

    full_board_bits = pieces_board_bits + turn_bits + castling_bits
    full_board_tensor = torch.tensor(full_board_bits, dtype=torch.float)

    return full_board_tensor

### Overweighed Data
Another problem that arise from our dataset can be seen in these examples:
```CSV
8/6k1/pp1p3p/3P2p1/2PN1rP1/1P6/P6P/6K1 w - - 1 39,+431
6R1/5p2/5k1P/8/4BKPn/r7/3p4/8 w - - 0 57,+7019
6R1/5p1P/5k2/8/4BKPn/r7/3p4/8 b - - 0 57,+6056
```

In some of the positions, one of the players is in a very big lead. These results could impact our training of the neural network much greater than what we want. 

For example, in one of the above examples, white leads in a 70 points! In a chess game, almost every player will win the game when he's up at even 15 points.

Therefore to minimize the affect of these big leads positions, we limit their values in the dataset to +15 or -15 (depends on if white/black is leading).


### Checkmate Positions 
Another problem that arose from our dataset can be seen in these examples:
```CSV
4Q1k1/2q2pp1/2b2b1p/p4B2/1pN5/1P5P/P4PP1/4R1K1 b - - 0 30,#+1,c6e8
3N1k2/3R4/1r5p/4p1p1/5p2/2P5/2P2PPP/6K1 b - - 0 27,#-2,b6b1
```
In the last 2 data entries, the evaluation is shown as "#-2" or "#+1" which means a checkmate can be done in a few moves.
"#-2" means black can do a checkmate in 2 moves, and "#+1" means white can do a checkmate in the next move.

These entries are unusual in their formula, and they can't be used easily for the training as the neural network is expected to give a number as the output. We can solve it by changing the score to fit our needs, if white is close to a checkmate we can give it a big lead, for example-
```CSV
4Q1k1/2q2pp1/2b2b1p/p4B2/1pN5/1P5P/P4PP1/4R1K1 b - - 0 30,#+1,c6e8
```
Can be changed to:
```CSV
4Q1k1/2q2pp1/2b2b1p/p4B2/1pN5/1P5P/P4PP1/4R1K1 b - - 0 30,+1500
```

In [6]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv(r'./res/chess_eval/tactic_evals.csv')

# Skip checkmate positions
df = df.loc[~df['Evaluation'].str.contains('#')]

df['Evaluation'] = df['Evaluation'].astype(int)
print(df)
very_big_lead_positions = df.query('Evaluation < -1500 or Evaluation > 1500')
print(very_big_lead_positions)
big_lead_percentage = len(very_big_lead_positions) / len(df)
print(f"Big lead positions has {len(very_big_lead_positions)} data points, and are {big_lead_percentage} of all dataset")



                                                       FEN  Evaluation  Move
4        6k1/pp6/3p4/2p1p3/2P1P1q1/1P1P2pP/P5P1/5K2 w -...         408  h3g4
5        6k1/pp6/3p4/2p1p3/2P1P1bq/1P1P2pP/P3Q1P1/5K2 w...         444  e2g4
6        6k1/pp6/3p4/2p1p3/2P1P1Qq/1P1P2pP/P5P1/5K2 b -...         428  h4g4
10       2k4r/pp3pp1/3N1p2/2pP1P1p/b3r1P1/P1P4P/2P5/R1R...         667  c8c7
12       6k1/pp6/3p4/2p1p3/2P1P1P1/1P1P2p1/P5P1/5K2 b -...         462  g8f7
...                                                    ...         ...   ...
2628214  3r4/3pk3/6q1/Qp1pPpP1/1PbPn3/4B3/5RKP/8 w - - ...       -1053  a5a3
2628215  r2qkbnr/5ppp/p1p1p3/3p4/3P2PP/2N5/PPb1QP2/R1B1...         215  h7h5
2628216  Q5b1/6pk/3q1pNp/1p1p3N/3P1P2/2P5/1P3KPP/8 b - ...         944  h7g6
2628217    3rb3/ppk1r2p/6p1/2P5/4B3/P7/6PP/3Q3K w - - 0 35        -692  d1c2
2628218  r6k/p1R2Qp1/1pP4p/3p4/3b1PPq/B6P/P5K1/8 w - - ...       -1138  f7g7

[2345781 rows x 3 columns]
                                                

## Training the Neural Network
Now that we know how to work with our data, I used pytorch to build and train the deep neural network.
But after a few trail and error and training for over 12 hours and even for a few full days, I realized that to get the network to better results, I must use a much better computing infrastructure.

### Google COLAB
Google offers a cloud computing environment for people all over the world to train their networks.
The infrastructure comes with a much better GPUs and helps to solve bottlenecks in the program.
**The transition to google colab allowed me to increase the training data size from around 300,000 to around 12,000,000.**

To get to 12 million and to use google colab, I had to do a lot of code changes and even use multiprocessing for loading the data faster.
The multiprocessing improved the loading time of the data to the GPU from over 4 hours to just few minutes!
And the result of the network with the much bigger dataset are much better accordingly.


In [ ]:
HIDDEN_LAYER_SIZE = 512
BITBOARD_SIZE = 773


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=BITBOARD_SIZE, out_features=HIDDEN_LAYER_SIZE),
            nn.ReLU(),
            nn.Linear(in_features=HIDDEN_LAYER_SIZE, out_features=HIDDEN_LAYER_SIZE),
            nn.ReLU(),
            nn.Linear(in_features=HIDDEN_LAYER_SIZE, out_features=1),
        )

    def forward(self, x):
        """The forward passing of an element through the network"""
        logits = self.linear_relu_stack(x)
        return logits


def _process_chunk(chunk, shared_list):
    logging.basicConfig(level=logging.INFO, format='%(processName)s: %(message)s')
    timestamp = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    logging.info(f"Starting to load chunk, starting loading on {timestamp}, chunk index: {chunk.index}")

    # TODO: Deal with ending positions and specifically mates (#). Replace by maximizing the player leading, '#+1' => +15, and '#-3' => -15.
    relevant_evaluations = chunk.loc[~chunk['Evaluation'].astype(str).str.contains('#')]
    all_fens = [] 
    all_evals = [] 
    for data in relevant_evaluations.iloc:
        try:
            fen = data['FEN']
            bitboard = fen_to_bitboard(fen)

            raw_eval_score = data['Evaluation']
            eval_score = int(raw_eval_score) / CENTIPAWN_UNIT
            eval_score = max(eval_score, -MAX_LEAD_SCORE)
            eval_score = min(eval_score, MAX_LEAD_SCORE)
        except ValueError as e:
            logging.warning(f"Found problematic item in dataset: fen - {fen}, score - {raw_eval_score}. Skipping")
            continue

        all_fens.append(bitboard)
        all_evals.append(eval_score)

    timestamp = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    logging.info(f"Finished to load chunk, finished loading on {timestamp}, chunk index: {chunk.index}")

    shared_list.append((all_fens, all_evals))
    logging.debug(f"Returning from multi process _process_chunk")
    return 

class ChessEvaluationsDataset(Dataset):
    DEFAULT_ITEM = ("r1bqk2r/pppp1ppp/2n1pn2/3P4/1b2P3/2N2Q2/PPP2PPP/R1B1KBNR b KQkq - 3 5", -0.6)

    def __init__(self, evaluations_file: str):
        csv_iter = pd.read_csv(evaluations_file, chunksize=10_000)

        timestamp = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
        logging.info(f"Starting to load dataset. Will load all to GPU, starting loading on {timestamp}")
        parsed_data = []

        with mp.Manager() as manager:
            processes = []
            shared_list = manager.list()
            for i, chunk in enumerate(csv_iter):
                if len(processes) >= CPU_CORES_COUNT:
                    processes.pop(0).join()
                
                p = mp.Process(target=_process_chunk, args=(chunk, shared_list))
                processes.append(p)
                p.start()

            for p in processes:
                p.join()

            parsed_data = list(shared_list)

        timestamp = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
        logging.info(f"Finished loading all dataset on {timestamp}")

        fens_arrays = [data[0] for data in parsed_data]
        evals_arrays = [data[1] for data in parsed_data]
        fens_data = np.concatenate(fens_arrays, axis=0)
        evals_data = np.concatenate(evals_arrays, axis=0)
        self.fens_tensor = torch.from_numpy(fens_data).to(DEVICE, dtype=torch.bool)
        self.evals_tensor = torch.from_numpy(evals_data).to(DEVICE, dtype=torch.int8).view(-1, 1)
        assert len(self.fens_tensor)==len(self.evals_tensor), "Unequal fens and evaluations, unexpected and should be debugged"

        logging.info(f"Finished parsing data!")

    def __len__(self):
        return len(self.fens_tensor)

    def __getitem__(self, index: int):
        return self.fens_tensor[index].to(torch.float32), self.evals_tensor[index].to(torch.float32)